In [ ]:
import tensorflow as tf
import numpy as np
import config
import os

In [ ]:
(training_images, training_labels) ,\
(validation_images, validation_labels) = \
tf.keras.datasets.cifar10.load_data()

In [ ]:

"""
  Pre-process the numpy array of images in the 'tf' mode.
  This method scales pixels between [-1.0, 1.0], sample-wise.

  Args:
    input_images: numpy.ndarray
      The input images to be pre-processed.

  Returns:  
    numpy.ndarray
      The pre-processed images scaled in the [-1.0, 1.0] range.
"""
def pre_process_fp32_images(input_images):
  if not isinstance(input_images, np.ndarray):
    raise TypeError("dataset images expected to be of type numpy.ndarray")
  input_images = input_images
  input_images = input_images.astype('float32')
  input_images /= 127.5 # scale a pixel (8 bit -> [0.0, 255.0]) in the range [0.0, 2.0]
  input_images -= 1.0 # adjust the range [0.0, 2.0] around 0, resulting in the range [-1.0, 1.0]
  return input_images

"""
  Pre-process the numpy array of images in the 'tf' mode (scaling pixels between [-1.0, 1.0]) sample-wise.
  Eventually, the pre-processed images are quantized to the specified quantization type
  and saved in the specified path.

  Args:
    input_images: numpy.ndarray
      The input images to be pre-processed.
    model: tf.lite.Interpreter
      The model to be used for retrieving quantization parameters.
    quantization_type: str ["int8", "uint8", "int16"]
      The quantization type to be used for quantizing the pre-processed images.

  Returns:  
    numpy.ndarray
      The pre-processed images quantized to the specified quantization type.
"""
def pre_process_images_for_quantized_models(input_images, model: tf.lite.Interpreter, quantization_type: str):
  if not isinstance(input_images, np.ndarray):
    raise TypeError("dataset images expected to be of type numpy.ndarray")
  
  input_details = model.get_input_details()[0]
  input_scale, input_zero_point = input_details["quantization"]
  
  if quantization_type == 'int8':
    quantized_images = tf.cast(pre_process_fp32_images(input_images) / input_scale + input_zero_point, tf.int8)
  elif quantization_type == 'uint8':
    quantized_images = tf.cast(pre_process_fp32_images(input_images) / input_scale + input_zero_point, tf.uint8)
  elif quantization_type == 'int16':
    quantized_images = pre_process_fp32_images(input_images)
  else:
    raise ValueError("quantization type not supported")
  
  return quantized_images

In [ ]:
for attr in dir(config):
  if attr.isupper():
    value = getattr(config, attr)
    if isinstance(value, str) and ("/" in value or "\\" in value):
      dir_path = os.path.dirname(value)
      if dir_path:
        os.makedirs(dir_path, exist_ok=True)

In [ ]:
"""
  Save the CIFAR-10 labels in the specified path.
"""
def save_cifar10_labels():
  np.save(config.TRAIN_LABELS_PATH, training_labels)
  np.save(config.VALIDATION_LABELS_PATH, validation_labels)

"""
  Save the first 2000 CIFAR-10 labels in the specified path.
"""
def save_first_2k_cifar10_labels():
  np.save(config.VALIDATION_LABELS_2K_PATH, validation_labels[:2000])

"""
  Save the first 500 CIFAR-10 labels in the specified path.
"""
def save_first_500_cifar10_labels():
  np.save(config.VALIDATION_LABELS_500_PATH, validation_labels[:500])

"""
  Save the CIFAR-10 preprocessed images (for fp32 models) in the specified path.
"""
def save_fp32_cifar10_data():
  train_X = pre_process_fp32_images(training_images)
  valid_X = pre_process_fp32_images(validation_images)
  np.save(config.FP32_TRAIN_SET_PREPROCESSED_PATH, train_X)
  np.save(config.FP32_VALIDATION_SET_PREPROCESSED_PATH, valid_X)
  
save_cifar10_labels()
save_fp32_cifar10_data()
save_first_2k_cifar10_labels()
save_first_500_cifar10_labels()

In [ ]:
"""
  Save the CIFAR-10 pre-processed and quantized images uint8, int8 for 
  alexnet quantized models, in the specified paths.
"""

def save_uint8_cifar10_x_alexnet_data():
  model = tf.lite.Interpreter(model_path=config.ALEXNET_U8_MODEL_PATH)
  model.allocate_tensors()
  _train_X = pre_process_images_for_quantized_models(training_images, model, 'uint8')
  valid_X = pre_process_images_for_quantized_models(validation_images, model, 'uint8')
  np.save(config.ALEXNET_U8_VALIDATION_SET_PREPROCESSED_PATH, valid_X)
  np.save(config.ALEXNET_U8_2K_VALIDATION_SET_PREPROCESSED_PATH, valid_X[:2000])
  
def save_int8_cifar10_x_alexnet_data():
  model = tf.lite.Interpreter(model_path=config.ALEXNET_I8_MODEL_PATH)
  model.allocate_tensors()
  _train_X = pre_process_images_for_quantized_models(training_images, model, 'int8')
  valid_X = pre_process_images_for_quantized_models(validation_images, model, 'int8')
  np.save(config.ALEXNET_I8_VALIDATION_SET_PREPROCESSED_PATH, valid_X)
  np.save(config.ALEXNET_I8_2K_VALIDATION_SET_PREPROCESSED_PATH, valid_X[:2000])

save_uint8_cifar10_x_alexnet_data()
save_int8_cifar10_x_alexnet_data()

In [ ]:
"""
  Save the CIFAR-10 pre-processed and quantized images uint8, int8 for 
  resnet50 quantized models, in the specified paths.
"""

def save_uint8_cifar10_x_resnet50_data():
  model = tf.lite.Interpreter(model_path=config.RESNET50_U8_MODEL_PATH)
  model.allocate_tensors()
  _train_X = pre_process_images_for_quantized_models(training_images, model, 'uint8')
  valid_X = pre_process_images_for_quantized_models(validation_images, model, 'uint8')
  np.save(config.RESNET50_U8_VALIDATION_SET_PREPROCESSED_PATH, valid_X)
  np.save(config.RESNET50_U8_2K_VALIDATION_SET_PREPROCESSED_PATH, valid_X[:2000])
  
def save_int8_cifar10_x_resnet50_data():
  model = tf.lite.Interpreter(model_path=config.RESNET50_I8_MODEL_PATH)
  model.allocate_tensors()
  _train_X = pre_process_images_for_quantized_models(training_images, model, 'int8')
  valid_X = pre_process_images_for_quantized_models(validation_images, model, 'int8')
  np.save(config.RESNET50_I8_VALIDATION_SET_PREPROCESSED_PATH, valid_X)
  np.save(config.RESNET50_I8_2K_VALIDATION_SET_PREPROCESSED_PATH, valid_X[:2000])

save_uint8_cifar10_x_resnet50_data()
save_int8_cifar10_x_resnet50_data()

In [ ]:
"""
  Save the CIFAR-10 pre-processed and quantized images uint8, int8 for 
  resnet152 quantized models, in the specified paths.
"""

def save_uint8_cifar10_x_resnet152_data():
  model = tf.lite.Interpreter(model_path=config.RESNET152_U8_MODEL_PATH)
  model.allocate_tensors()
  _train_X = pre_process_images_for_quantized_models(training_images, model, 'uint8')
  valid_X = pre_process_images_for_quantized_models(validation_images, model, 'uint8')
  np.save(config.RESNET152_U8_VALIDATION_SET_PREPROCESSED_PATH, valid_X)
  np.save(config.RESNET152_U8_2K_VALIDATION_SET_PREPROCESSED_PATH, valid_X[:2000])
  
def save_int8_cifar10_x_resnet152_data():
  model = tf.lite.Interpreter(model_path=config.RESNET152_I8_MODEL_PATH)
  model.allocate_tensors()
  _train_X = pre_process_images_for_quantized_models(training_images, model, 'int8')
  valid_X = pre_process_images_for_quantized_models(validation_images, model, 'int8')
  np.save(config.RESNET152_I8_VALIDATION_SET_PREPROCESSED_PATH, valid_X)
  np.save(config.RESNET152_I8_2K_VALIDATION_SET_PREPROCESSED_PATH, valid_X[:2000])

save_uint8_cifar10_x_resnet152_data()
save_int8_cifar10_x_resnet152_data()

In [ ]:
"""
  Save the CIFAR-10 pre-processed and quantized images uint8, int8 for 
  vgg16 quantized models, in the specified paths.
"""

def save_uint8_cifar10_x_vgg16_data():
  model = tf.lite.Interpreter(model_path=config.VGG16_U8_MODEL_PATH)
  model.allocate_tensors()
  _train_X = pre_process_images_for_quantized_models(training_images, model, 'uint8')
  valid_X = pre_process_images_for_quantized_models(validation_images, model, 'uint8')
  np.save(config.VGG16_U8_VALIDATION_SET_PREPROCESSED_PATH, valid_X)
  np.save(config.VGG16_U8_2K_VALIDATION_SET_PREPROCESSED_PATH, valid_X[:2000])
  
def save_int8_cifar10_x_vgg16_data():
  model = tf.lite.Interpreter(model_path=config.VGG16_I8_MODEL_PATH)
  model.allocate_tensors()
  _train_X = pre_process_images_for_quantized_models(training_images, model, 'int8')
  valid_X = pre_process_images_for_quantized_models(validation_images, model, 'int8')
  np.save(config.VGG16_I8_VALIDATION_SET_PREPROCESSED_PATH, valid_X)
  np.save(config.VGG16_I8_2K_VALIDATION_SET_PREPROCESSED_PATH, valid_X[:2000])

save_uint8_cifar10_x_vgg16_data()
save_int8_cifar10_x_vgg16_data()

In [ ]:
"""
  Save the CIFAR-10 pre-processed and quantized images uint8, int8 for 
  mobilenet quantized models, in the specified paths.
"""

def save_uint8_cifar10_x_mobilenet_data():
  model = tf.lite.Interpreter(model_path=config.MOBILENET_U8_MODEL_PATH)
  model.allocate_tensors()
  _train_X = pre_process_images_for_quantized_models(training_images, model, 'uint8')
  valid_X = pre_process_images_for_quantized_models(validation_images, model, 'uint8')
  np.save(config.MOBILENET_U8_VALIDATION_SET_PREPROCESSED_PATH, valid_X)
  np.save(config.MOBILENET_U8_2K_VALIDATION_SET_PREPROCESSED_PATH, valid_X[:2000])
  
def save_int8_cifar10_x_mobilenet_data():
  model = tf.lite.Interpreter(model_path=config.MOBILENET_I8_MODEL_PATH)
  model.allocate_tensors()
  _train_X = pre_process_images_for_quantized_models(training_images, model, 'int8')
  valid_X = pre_process_images_for_quantized_models(validation_images, model, 'int8')
  np.save(config.MOBILENET_I8_VALIDATION_SET_PREPROCESSED_PATH, valid_X)
  np.save(config.MOBILENET_I8_2K_VALIDATION_SET_PREPROCESSED_PATH, valid_X[:2000])

save_uint8_cifar10_x_mobilenet_data()
save_int8_cifar10_x_mobilenet_data()

In [ ]:
"""
  Save the CIFAR-10 pre-processed and quantized images uint8, int8 for 
  mobilenetV2 quantized models, in the specified paths.
"""

def save_uint8_cifar10_x_mobilenetV2_data():
  model = tf.lite.Interpreter(model_path=config.MOBILENETV2_U8_MODEL_PATH)
  model.allocate_tensors()
  _train_X = pre_process_images_for_quantized_models(training_images, model, 'uint8')
  valid_X = pre_process_images_for_quantized_models(validation_images, model, 'uint8')
  np.save(config.MOBILENETV2_U8_VALIDATION_SET_PREPROCESSED_PATH, valid_X)
  np.save(config.MOBILENETV2_U8_2K_VALIDATION_SET_PREPROCESSED_PATH, valid_X[:2000])
  
def save_int8_cifar10_x_mobilenetV2_data():
  model = tf.lite.Interpreter(model_path=config.MOBILENETV2_I8_MODEL_PATH)
  model.allocate_tensors()
  _train_X = pre_process_images_for_quantized_models(training_images, model, 'int8')
  valid_X = pre_process_images_for_quantized_models(validation_images, model, 'int8')
  np.save(config.MOBILENETV2_I8_VALIDATION_SET_PREPROCESSED_PATH, valid_X)
  np.save(config.MOBILENETV2_I8_2K_VALIDATION_SET_PREPROCESSED_PATH, valid_X[:2000])

save_uint8_cifar10_x_mobilenetV2_data()
save_int8_cifar10_x_mobilenetV2_data()

In [ ]:
"""
  Save the CIFAR-10 pre-processed and quantized images uint8, int8 for 
  efficientnetB0 quantized models, in the specified paths.
"""

def save_uint8_cifar10_x_efficientnetB0_data():
  model = tf.lite.Interpreter(model_path=config.EFFICIENTNETB0_U8_MODEL_PATH)
  model.allocate_tensors()
  _train_X = pre_process_images_for_quantized_models(training_images, model, 'uint8')
  valid_X = pre_process_images_for_quantized_models(validation_images, model, 'uint8')
  np.save(config.EFFICIENTNETB0_U8_VALIDATION_SET_PREPROCESSED_PATH, valid_X)
  np.save(config.EFFICIENTNETB0_U8_2K_VALIDATION_SET_PREPROCESSED_PATH, valid_X[:2000])
  
def save_int8_cifar10_x_efficientnetB0_data():
  model = tf.lite.Interpreter(model_path=config.MOBILENETV2_I8_MODEL_PATH)
  model.allocate_tensors()
  _train_X = pre_process_images_for_quantized_models(training_images, model, 'int8')
  valid_X = pre_process_images_for_quantized_models(validation_images, model, 'int8')
  np.save(config.EFFICIENTNETB0_I8_VALIDATION_SET_PREPROCESSED_PATH, valid_X)
  np.save(config.EFFICIENTNETB0_I8_2K_VALIDATION_SET_PREPROCESSED_PATH, valid_X[:2000])

save_uint8_cifar10_x_efficientnetB0_data()
save_int8_cifar10_x_efficientnetB0_data()